In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#for dirname, _, filenames in os.walk('../input/humpback-whale-identification/train'):
#    for filename in filenames:
#        print(filename)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
import torch
torch.cuda.is_available()

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('../input/humpback-whale-identification/train.csv')
df.head()

In [ ]:
tfm = rand_pad(4, 150)
tfms = get_transforms(max_lighting =0.1, max_zoom =1.05, max_warp = 0.2, do_flip = True, max_rotate=25)


In [ ]:
np.random.seed(52)
src = (ImageList.from_csv('../input/humpback-whale-identification/',
                         'train.csv', folder = 'train').split_by_rand_pct(0.2).label_from_df(label_delim=' '))

In [ ]:
src

In [ ]:
data = (src.transform(tfms, size=(150,300))
        .databunch().normalize(imagenet_stats))

In [ ]:
data.show_batch(rows =3, figsize=(12,9))

In [ ]:
arch = models.resnet101

In [ ]:
learn = cnn_learner(data, arch, metrics=fbeta)

In [ ]:

learn.model_dir="/kaggle/working"

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 1e-02

In [ ]:
learn.fit_one_cycle(10, slice(lr), wd =1e-02)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('whale')

In [ ]:
learn.load('whale')

In [ ]:
learn.unfreeze()

In [ ]:
learn.detach()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, slice(7e-4, (7e-4)/5), wd = 0.1)

In [ ]:
    learn.recorder.plot_losses()

In [ ]:
learn.save('stage-2-rn50')

In [ ]:
learn.load('stage-2-rn50')

In [ ]:
data = (src.transform(tfms, size = (350,700))
       .databunch().normalize(imagenet_stats))

learn.data = data
data.train_ds[0][0].shape

In [ ]:
learn.to_fp16()

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 1e-3

In [ ]:
learn.fit_one_cycle(5,slice(lr, lr/5))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-1 350-rn50')

In [ ]:
learn.load('stage-1 350-rn50')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 1e-4

In [ ]:
learn.fit_one_cycle(10,slice(lr, lr/5), wd = 1e-03)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-2 350-rn50')

In [ ]:
learn.load('stage-2 350-rn50')

In [ ]:
learn.export('/kaggle/working/export.pkl')

In [ ]:
learn.recorder.plot()

# Test

In [ ]:
test = ImageList.from_folder('../input/humpback-whale-identification/test')
len(test)

In [ ]:
learn = load_learner('/kaggle/working/', test=test).to_fp16()
preds, _ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
temp = torch.topk(preds, 5)

In [ ]:
labelled_preds = [' '.join([learn.data.classes[p] for i,p in enumerate(pred)]) for pred in temp.indices]

In [ ]:
learn.data.test_ds.items

In [ ]:
fnames = [f.name for f in learn.data.test_ds.items]

In [ ]:
df = pd.DataFrame({'image_name':fnames, 'tags':labelled_preds}, columns=['image_name', 'tags'])

In [ ]:
df.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
df1 = pd.DataFrame({'Image':fnames, 'id':labelled_preds}, columns=['Image', 'id'])
df1.to_csv('/kaggle/working/submission1.csv', index=False)